In [3]:
import pandas as pd
import s2cell
import s2sphere

# Reduce the number for csv

In [7]:
df_sample_csv=pd.read_csv('../00-data/sample/meta_shard_0.csv')
df_sample_csv=df_sample_csv[0:150]
df_sample_csv.to_csv('../00-data/sample/meta_shard_0.csv',index=False)

In [81]:
df_sample_csv['geohash_2'] = df_sample_csv.apply(lambda x: s2cell.lat_lon_to_cell_id(x.lat,x.lon,2),axis=1)
df_sample_csv['geohash_3'] = df_sample_csv.apply(lambda x: s2cell.lat_lon_to_cell_id(x.lat,x.lon,3),axis=1)

In [82]:
df_sample_csv['count']=1

In [89]:
df_sample_csv.sample()

,img,lat,lon,data,count,geohash_3
105,60/bb/4686785221.jpg,51.5,7.0,ffd8ffe000104a46494600010100000100010000ffdb00...,1,5170132372221329408


In [90]:
# zoom_n1 = df_sample_csv[['geohash_2','count']]
zoom_n1 = df_sample_csv[['geohash_3','count']]
# zoom_n1=zoom_n1.groupby(by='geohash_2').count().reset_index()
zoom_n1=zoom_n1.groupby(by='geohash_3').count().reset_index()

In [91]:
zoom_n1

,geohash_3,count
0,954763121002545152,8
1,1351079888211148800,14
2,5170132372221329408,91
3,5206161169240293376,37


In [92]:
# if count is > 5 then proceed to creation of geohash_i+1
# if geohash_i-1 exist delete it
df_sample_csv['geohash_4'] = df_sample_csv.apply(lambda x: s2cell.lat_lon_to_cell_id(x.lat,x.lon,4),axis=1)
zoom_n2 = df_sample_csv[['geohash_4','count']]
zoom_n2=zoom_n2.groupby(by='geohash_4').count().reset_index()

In [93]:
zoom_n2

,geohash_4,count
0,959266720629915648,8
1,1346576288583778304,6
2,1355583487838519296,8
3,5156621573339217920,36
4,5165628772593958912,14
5,5174635971848699904,17
6,5183643171103440896,24
7,5192650370358181888,3
8,5219671968122404864,34


In [97]:
zoom_n2['geohash_4'][zoom_n2['count']<5]

7    5192650370358181888
Name: geohash_4, dtype: int64

In [101]:
df_sample_csv

,img,lat,lon,data,count,geohash_3,geohash_4,cellid
0,a6/c4/3038144879.jpg,48.9,2.3,ffd8ffe000104a46494600010100000100010000ffdb00...,1,5170132372221329408,5183643171103440896,_
1,a7/00/2633379373.jpg,45.9,0.2,ffd8ffe000104a46494600010100000100010000ffdb00...,1,5170132372221329408,5183643171103440896,_
2,03/2e/4493999629.jpg,50.9,6.9,ffd8ffe000104a46494600010100000100010000ffdb00...,1,5170132372221329408,5165628772593958912,_
3,c6/12/3073882997.jpg,51.5,7.5,ffd8ffe000104a46494600010100000100010000ffdb00...,1,5170132372221329408,5165628772593958912,_
4,0d/1d/3586372001.jpg,43.9,5.2,ffd8ffe000104a46494600010100000100010000ffdb00...,1,1351079888211148800,1355583487838519296,_
...,...,...,...,...,...,...,...,...
145,26/30/4883130713.jpg,51.5,-0.1,ffd8ffe000104a46494600010100000100010000ffdb00...,1,5206161169240293376,5219671968122404864,_
146,bd/f0/2340575216.jpg,48.9,2.3,ffd8ffe000104a46494600010100000100010000ffdb00...,1,5170132372221329408,5183643171103440896,_
147,98/b9/5657256708.jpg,46.3,-1.1,ffd8ffe000104a46494600010100000100010000ffdb00...,1,5206161169240293376,5192650370358181888,_
148,0d/89/8234978054.jpg,49.8,6.6,ffd8ffe000104a46494600010100000100010000ffdb00...,1,5170132372221329408,5156621573339217920,_


In [107]:
df_sample_csv['geohash_4'] zoom_n2['geohash_4'][zoom_n2['count']<5]

7    5192650370358181888
Name: geohash_4, dtype: int64

In [117]:
if df_sample_csv['geohash_4'].isin(zoom_n2['geohash_4'][zoom_n2['count']<5]
                                ):
    print('I did it')
    df_sample_csv['cellid']=df_sample_csv['geohash_3']

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [110]:
df_sample_csv.sort_values(by='cellid')

,img,lat,lon,data,count,geohash_3,geohash_4,cellid
0,a6/c4/3038144879.jpg,48.9,2.3,ffd8ffe000104a46494600010100000100010000ffdb00...,1,5170132372221329408,5183643171103440896,_
96,4b/b4/155851248.jpg,48.9,2.3,ffd8ffe000104a46494600010100000100010000ffdb00...,1,5170132372221329408,5183643171103440896,_
97,76/cb/4203291268.jpg,49.0,8.4,ffd8ffe000104a46494600010100000100010000ffdb00...,1,5170132372221329408,5156621573339217920,_
98,ed/dc/4833052943.jpg,51.5,-2.6,ffd8ffe000104a46494600010100000100010000ffdb00...,1,5206161169240293376,5219671968122404864,_
99,3b/93/713825.jpg,51.8,4.7,ffd8ffe000104a46494600010100000100010000ffdb00...,1,5170132372221329408,5174635971848699904,_
...,...,...,...,...,...,...,...,...
51,0e/1e/3519764726.jpg,51.5,6.7,ffd8ffe000104a46494600010100000100010000ffdb00...,1,5170132372221329408,5165628772593958912,_
52,d7/e3/2928047852.jpg,50.0,8.1,ffd8ffe000104a46494600010100000100010000ffdb00...,1,5170132372221329408,5165628772593958912,_
53,02/c5/5222001472.jpg,49.7,-2.2,ffd8ffe000104a46494600010100000100010000ffdb00...,1,5206161169240293376,5192650370358181888,_
55,fd/dc/6537106387.jpg,45.9,9.9,ffd8ffe000104a46494600010100000100010000ffdb00...,1,5170132372221329408,5156621573339217920,_


In [72]:
zoom_n1=zoom_n1[zoom_n1['count'].isin(range(5))]

In [74]:
zoom_n2=zoom_n2[~zoom_n2['geohash_6'].isin(zoom_n1['geohash_5'])]

In [76]:
zoom_n2=zoom_n2[zoom_n2['count'].isin(range(5))]

In [77]:
zoom_n1

,geohash_5,count
0,955889020909387776,1
1,958140820723073024,4
2,960392620536758272,3
3,1343198588863250432,1
4,1345450388676935680,3
5,1347702188490620928,2
10,5159999273059745792,4
15,5182517271196598272,2
17,5187020870823968768,2
18,5189272670637654016,1


In [79]:
zoom_n2

,geohash_6,count
0,956733445839519744,1
1,957296395792941056,1
2,958985245653204992,3
3,959548195606626304,1
4,960674095513468928,2
5,1343480063839961088,1
6,1344605963746803712,1
7,1346294813607067648,2
8,1347983663467331584,1
9,1348546613420752896,1


In [80]:
df_sample_csv['cell_id']=

,img,lat,lon,data,geohash_5,geohash_6,count
0,a6/c4/3038144879.jpg,48.9,2.3,ffd8ffe000104a46494600010100000100010000ffdb00...,5180265471382913024,5181109896313044992,1
1,a7/00/2633379373.jpg,45.9,0.2,ffd8ffe000104a46494600010100000100010000ffdb00...,5187020870823968768,5187865295754100736,1
2,03/2e/4493999629.jpg,50.9,6.9,ffd8ffe000104a46494600010100000100010000ffdb00...,5169006472314486784,5169850897244618752,1
3,c6/12/3073882997.jpg,51.5,7.5,ffd8ffe000104a46494600010100000100010000ffdb00...,5169006472314486784,5168162047384354816,1
4,0d/1d/3586372001.jpg,43.9,5.2,ffd8ffe000104a46494600010100000100010000ffdb00...,1354457587931676672,1354176112954966016,1
...,...,...,...,...,...,...,...
145,26/30/4883130713.jpg,51.5,-0.1,ffd8ffe000104a46494600010100000100010000ffdb00...,5220797868029247488,5221642292959379456,1
146,bd/f0/2340575216.jpg,48.9,2.3,ffd8ffe000104a46494600010100000100010000ffdb00...,5180265471382913024,5181109896313044992,1
147,98/b9/5657256708.jpg,46.3,-1.1,ffd8ffe000104a46494600010100000100010000ffdb00...,5189272670637654016,5190117095567785984,1
148,0d/89/8234978054.jpg,49.8,6.6,ffd8ffe000104a46494600010100000100010000ffdb00...,5157747473246060544,5158028948222771200,1
